In [ ]:
# import sys
# import os
# from os import path

# # sys.path.append('C:\\Users\\Administrator\\Desktop\\风控产品\\risk_project')
# sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
from risk_models import *

class ScoreFn(object):
    def __init__(self, org_code, params, base_time, child_task_id):
        self.model_code = 'FINANCE'
        self.child_model_code = 'SCORE'
        self.child_task_id = child_task_id
        Risk_logger(child_task_id=self.child_task_id, exec_status=None)

        # 参数读取
        self.org_code = org_code
        self.score_weight = json.loads(params)['score_weight']
        
    def score_fn(self):  
        # 读数
        sql=f"""\
        select sum(SCORE) from {TableList.BD_RISK_RESULT_FINANCE_FN1.value} WHERE ISCURRENT = 1 AND ORG_CODE LIKE '{self.org_code}' \
        """
        fn1_result=Read_Oracle().read_oracle(sql=sql,database='dbods')

        sql=f"""\
        select sum(SCORE) from {TableList.BD_RISK_RESULT_FINANCE_FN2.value} WHERE ISCURRENT = 1 AND ORG_CODE LIKE '{self.org_code}' \
        """
        fn2_result=Read_Oracle().read_oracle(sql=sql,database='dbods')

        x = [fn1_result.iloc[0,0],fn2_result.iloc[0,0]]

        total_score = 0
        for i in range(len(x)):
            if x[i]:
                total_score = total_score + (100 + x[i])*self.score_weight[i]
            else:
                total_score = total_score + 0

        df_result = pd.DataFrame()
        now = datetime.datetime.strftime(datetime.datetime.now(), format='%Y-%m-%d %H:%M:%S')
        df_result = df_result.append([{'ID':0,'ORG_CODE':self.org_code,'CHECK_TIME':now}],ignore_index = True)
        df_result['CHECK_TIME']= pd.to_datetime(df_result['CHECK_TIME'], format='%Y-%m-%d %H:%M:%S')
        df_result['SCORE'] = total_score
        df_result['MODEL_CODE'] = self.model_code

        Write_Oracle().write_oracle(f'{TableList.BD_RISK_CORP_SCORE_DISPLAY.value}',df_result,org_code=org_code,alarm = ['FINANCE',''])

    def run_score_fn(self):
        try:
            self.score_fn()
            exec_status = 1
        except:
            logger.exception('model execution error')
            exec_status = 0
        finally:
            Risk_logger(child_task_id=self.child_task_id, exec_status=exec_status).write_log()


if __name__ == '__main__':
    # 读取传入的 child_task_id，仅适用于py调用,先备注掉
#     child_task_id = sys.argv[1]
    child_task_id = '0001_0006'
    org_code, param_json, basetime = read_log_table(child_task_id)
    ScoreFn(org_code, params=param_json, base_time = basetime, child_task_id=child_task_id).run_score_fn()

In [1]:
import sys
import os
from os import path

sys.path.append('C:\\Users\\Administrator\\Desktop\\风控产品\\risk_project')
# sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
from risk_models import *

In [2]:
# 读数
sql=f"""\
select sum(SCORE) from {TableList.BD_RISK_RESULT_WAREHOUSE_WH1.value} WHERE ISCURRENT = 1  \
"""
wh1_result=Read_Oracle().read_oracle(sql=sql,database='dbods')

sql=f"""\
select sum(SCORE) from {TableList.BD_RISK_RESULT_WAREHOUSE_WH2.value} WHERE ISCURRENT = 1 \
"""
wh2_result=Read_Oracle().read_oracle(sql=sql,database='dbods')

x = [wh1_result.iloc[0,0],wh2_result.iloc[0,0]]

total_score = 0
score_weight = [0.5,0.5]

for i in range(len(x)):
    if x[i]:
        total_score = total_score + (100 + x[i])*score_weight[i]
    else:
        total_score = total_score + 0

df_result = pd.DataFrame()
now = datetime.datetime.strftime(datetime.datetime.now(), format='%Y-%m-%d %H:%M:%S')
df_result = df_result.append([{'ID':0,'ORG_CODE':000000000,'CHECK_TIME':now}],ignore_index = True)
df_result['CHECK_TIME']= pd.to_datetime(df_result['CHECK_TIME'], format='%Y-%m-%d %H:%M:%S')
df_result['SCORE'] = total_score
df_result['MODEL_CODE'] = 'WAREHOUSE'

# Write_Oracle().write_oracle(f'{TableList.BD_RISK_CORP_SCORE_DISPLAY.value}',df_result,org_code=org_code,alarm = ['FINANCE',''])

2021-06-08 09:33:11.082 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 2.209s
2021-06-08 09:33:11.221 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.128s


In [18]:
sql=f"""\
select sum(SCORE) from {TableList.BD_RISK_RESULT_WAREHOUSE_WH2.value} WHERE ISCURRENT = 1 AND ORG_CODE LIKE '11111' \
"""
wh3_result=Read_Oracle().read_oracle(sql=sql,database='dbods')

2021-06-08 09:41:06.342 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 5.858s


In [19]:
wh3_result

,SUM(SCORE)
0,None


In [35]:
total_score = 0
score_weight = [0.5,0.5]

for i in range(len(x)):
    if pd.isnull(x[i]):
        total_score = total_score + 0
    elif x[i] < -100 :
        total_score = total_score + (100 - 100 )*score_weight[i]
    elif x[i] > 0 :
        total_score = total_score + (100 + 0 )*score_weight[i]
    else:
        total_score = total_score + (100 + x[i])*score_weight[i]
    print(total_score)

total_score = round(total_score,2)
print(total_score)

26.4444265
39.22371425
39.22


In [34]:
x = [-47.111147,-74.4414245]

In [24]:
(-pd.isnull(wh3_result.iloc[0,0]))

-1

In [26]:
-False

0

In [38]:
import sys, os
from os import path
# sys.path.append('/root/bdrisk/risk_project')
# sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
sys.path.append('C:\\Users\\Administrator\\Desktop\\风控产品\\risk_project')
# sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
from risk_models import *

class ScoreWh(object):
    def __init__(self, org_code, params, base_time, child_task_id):
        self.model_code = 'WAREHOUSE'
        self.child_model_code = 'SCORE'
        self.child_task_id = child_task_id
        Risk_logger(child_task_id=self.child_task_id, exec_status=None)

        # 参数读取
        self.org_code = org_code
        self.score_weight = [0.5,0.5]
        
    def score_wh(self):  
        # 读数
        sql=f"""\
        select sum(SCORE) from {TableList.BD_RISK_RESULT_WAREHOUSE_WH1.value} WHERE ISCURRENT = 1  \
        """
        wh1_result=Read_Oracle().read_oracle(sql=sql,database='dbods')

        sql=f"""\
        select sum(SCORE) from {TableList.BD_RISK_RESULT_WAREHOUSE_WH2.value} WHERE ISCURRENT = 1 \
        """
        wh2_result=Read_Oracle().read_oracle(sql=sql,database='dbods')

        x = [wh1_result.iloc[0,0],wh2_result.iloc[0,0]]

        total_score = 0
        for i in range(len(x)):
            if pd.isnull(x[i]):
                total_score = total_score + 0
            elif x[i] < -100 :
                total_score = total_score + (100 - 100 )*self.score_weight[i]
            elif x[i] > 0 :
                total_score = total_score + (100 + 0 )*self.score_weight[i]
            else:
                total_score = total_score + (100 + x[i])*self.score_weight[i]
                
        total_score = round(total_score,2)
        df_result = pd.DataFrame()
        now = datetime.datetime.strftime(datetime.datetime.now(), format='%Y-%m-%d %H:%M:%S')
        df_result = df_result.append([{'ID':0,'ORG_CODE':self.org_code,'CHECK_TIME':now}],ignore_index = True)
        df_result['CHECK_TIME']= pd.to_datetime(df_result['CHECK_TIME'], format='%Y-%m-%d %H:%M:%S')
        df_result['SCORE'] = total_score
        df_result['MODEL_CODE'] = self.model_code
        print(df_result)

        Write_Oracle().write_oracle(f'{TableList.BD_RISK_CORP_SCORE_DISPLAY.value}',df_result,org_code=self.org_code,alarm = ['WAREHOUSE',''])

    def run_score_wh(self):
        try:
            self.score_wh()
            exec_status = 1
        except:
            logger.exception('model execution error')
            exec_status = 0
        finally:
            Risk_logger(child_task_id=self.child_task_id, exec_status=exec_status).write_log()


if __name__ == '__main__':
    # 读取传入的 child_task_id，仅适用于py调用,先备注掉
#     child_task_id = sys.argv[1]
    child_task_id = '0001_0005'
    org_code, param_json, basetime = read_log_table(child_task_id)
    org_code = '91310115688774070B'
    ScoreWh(org_code, params=param_json, base_time = basetime, child_task_id=child_task_id).run_score_wh()

2021-06-08 10:11:41.480 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.223s
2021-06-08 10:11:41.593 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.110s
2021-06-08 10:11:41.707 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.090s


   ID            ORG_CODE          CHECK_TIME  SCORE MODEL_CODE
0   0  91310115688774070B 2021-06-08 10:11:41  99.11  WAREHOUSE


2021-06-08 10:11:44.676 | INFO     | risk_models.config.write_config.write_func:write_oracle:179 - Processing... Writing 1 rows into database
2021-06-08 10:11:44.702 | INFO     | risk_models.config.write_config.write_func:write_oracle:186 - Insert data into BD_RISK_CORP_SCORE_DISPLAY successfully! Total write time spent 3.090s
2021-06-08 10:11:44.718 | INFO     | risk_models.config.log_config.log_func:gen_log_json:37 - updating child_task_id:0001_0005 log msg into BD_RISK_MODEL_LOG successfully!
2021-06-08 10:11:44.971 | INFO     | risk_models.config.write_config.write_log:write_oracle:52 - Updated child_task_id: 0001_0005 log msg into BD_RISK_MODEL_LOG successfully!
